In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Package for handling logistic regression
from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# About the data

Once again, we'll be going back to our crime data, this time for the state of Virginia. As a reminder, our dataset contains data about each city, along with its 
1. population, 
2. violent crime rate, 
3. murder, 
4. rape, 
5. robbery, 
6. aggravated assault, 
7. property crime, 
8. burglary,
9. larceny,
10. motor vehicle theft,
11. arson

# Objective

Our objective for this project is to determine simply whether a city has a severe problem with property crime or not. For convenience sake, I have defined severe property crime as having over 100 incidents per year.

In [2]:
# Importing our data
data = pd.read_excel('Data\\table_8_offenses_known_to_law_enforcement_virginia_by_city_2013.xls', 
                     encoding='latin1', skiprows=list(range(4)))

# This column contains no data, so we're going to remove it
data.drop(['Rape\n(legacy\ndefinition)2'], axis = 1, inplace = True)

# Cleaning up our column names
data.columns = data.columns.str.replace('\n', ' ')
data = data.rename(index = str, columns={'Rape (revised definition)1': 'Rape'})
data.drop(['City'], axis = 1, inplace = True)
data = data.dropna()
data = data[:-1]

In [3]:
data.head()

,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
0,8186.000,10.000,0.000,3.000,1.000,6.000,233.000,20.000,198.000,15.000,4.000
1,148519.000,258.000,5.000,21.000,118.000,114.000,2967.000,249.000,2427.000,291.000,13.000
2,3486.000,8.000,0.000,0.000,2.000,6.000,56.000,4.000,52.000,0.000,0.000
3,2223.000,2.000,0.000,2.000,0.000,0.000,27.000,6.000,19.000,2.000,0.000
4,1728.000,12.000,0.000,2.000,2.000,8.000,77.000,25.000,51.000,1.000,0.000


In [4]:
data.describe()

,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
count,148.000,148.000,148.000,148.000,148.000,148.000,148.000,148.000,148.000,148.000,148.000
mean,20170.378,65.014,1.304,6.953,20.365,36.392,612.554,100.926,478.953,32.676,3.277
std,54170.809,201.049,4.796,18.482,72.990,111.979,1729.381,312.329,1325.700,115.662,11.514
min,112.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,1438.500,1.000,0.000,0.000,0.000,0.000,19.000,3.000,16.750,0.000,0.000
50%,3347.500,5.000,0.000,1.000,0.000,4.000,62.500,9.000,54.500,2.000,0.000
75%,12460.250,25.250,0.000,4.000,6.000,13.250,272.000,27.250,251.500,11.000,1.000
max,450687.000,1418.000,37.000,140.000,624.000,842.000,11226.000,2039.000,9374.000,938.000,99.000


In [5]:
# Create a dataframe where property crimes exceed 100
y = pd.DataFrame()
y['Property crime'] = np.where(data['Property crime'] < 100, 0, 1)

# Drop property crime from the dataframe since that's the 
# variable that we want to predict
data.drop(['Property crime'], axis = 1, inplace = True)

# Construct the test and training sets
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.5)

In [14]:
# Builds the regression model
def regressionModelBuilder(X, y):
    lr = LogisticRegression(C=1e9)
    fit = lr.fit(X, y)

    # Display
    print('Coefficients')
    print(fit.coef_)
    print(fit.intercept_)
    pred_y_sklearn = lr.predict(X)

    print('\n Accuracy by severe crime prediction')
    print(pd.crosstab(pred_y_sklearn, y))

    print('\n Percentage accuracy')
    print(lr.score(X, y))

In [15]:
regressionModelBuilder(X_test, y_test)

Coefficients
[[-3.08024216e-04 -1.39131541e-01  4.07236932e-01 -5.62481085e-01
  -1.87465582e-01  2.03578193e-01 -1.19568315e-01  8.51612163e-02
   2.52449988e-01  4.43015594e-01]]
[-4.02204402]

 Accuracy by severe crime prediction
col_0  (P, r, o, p, e, r, t, y,  , c, r, i, m, e)
row_0                                            
0                                              42
1                                              32

 Percentage accuracy
0.9864864864864865


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
regressionModelBuilder(X_train, y_test)

Coefficients
[[ 3.98198775e-05  1.57611850e-01  5.75140929e-01 -7.36192220e-02
  -2.10648672e-01 -1.33261185e-01 -1.38372113e-03 -5.20306212e-03
   7.70072077e-03  3.58005452e-02]]
[-0.32902036]

 Accuracy by severe crime prediction
col_0  (P, r, o, p, e, r, t, y,  , c, r, i, m, e)
row_0                                            
0                                              64
1                                              10

 Percentage accuracy
0.6351351351351351


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
# Ridge regression model
ridgeregr = linear_model.Ridge(alpha=10, fit_intercept=False) 
ridgeregr.fit(X_train, y_train)
print('R² for the model with few features:')
print(ridgeregr.score(X_train, y_train))
origparams = ridgeregr.coef_[0]
print('\nParameter estimates for the model with few features:')
print(origparams)

R² for the model with few features:
0.15809708398626787

Parameter estimates for the model with few features:
[ 2.80005177e-05  3.22625109e-02  6.74854527e-02  1.05510401e-02
 -8.49576113e-03 -3.72782207e-02 -1.85517693e-03 -2.12019224e-04
 -1.62817585e-02 -4.87080938e-02]


In [18]:
# Lasso Regression Model
lass = linear_model.Lasso(alpha=.35)
lassfit = lass.fit(X_train, y_train)
print('R² for the model with few features:')
print(lass.score(X_train, y_train))
origparams = np.append(lassfit.coef_, lassfit.intercept_)
print('\nParameter estimates for the model with few features:')
print(origparams)

R² for the model with few features:
0.2784206081950641

Parameter estimates for the model with few features:
[ 6.59563030e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.66037952e-04 -4.65265293e-03  1.16415237e-04  5.07058503e-04
 -1.83025585e-03 -0.00000000e+00  3.16046805e-01]
